Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import catboost as cb
from datetime import datetime
import seaborn as sns

In [2]:
data_train = pd.read_csv("./data/train.csv")
data_test = pd.read_csv("./data/test.csv")
data_webstat = pd.read_csv("./data/t1_webstat.csv")

is_callcenter - был ли совершен звонок в коллцентр по итогам сессии (нужно предсказать)

order_id - id заказа

create_time - время создания заказа

good_id - id товара

price - цена товара

utm_medium - тип трафика

utm_source - источник трафика

sessionkey_id - id сессии пользователя

category_id - id категории товара

parent_id - id родительской категории для категории товара

root_id - id "базовой" категории товара

rating_value - рейтинг товара

rating_count - число оценок товара

description_length - длина описания товара

goods_qty - число товаров в заказе

model_id - ???

is_moderated - ???

pics_qty - ???

model_create_time - ???

sessionkey_id - id сессии пользователя

date_time - timestamp захода на страницу определенного типа

page_type - тип просмотренной страницы

pageview_number - порядковый номер страницы среди всех страниц, отсортированных по времени посещения внутри сессии

pageview_duration_sec - время просмотра страницы

product_in_sale - ???

In [3]:
data_train["create_time_ts"] = None
data_test["create_time_ts"] = None
data_train["model_create_time_ts"] = None
data_test["model_create_time_ts"] = None

from datetime import datetime
# convert string to date
data_train.create_time_ts = data_train.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.create_time_ts = data_test.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

data_train.loc[~data_train.model_create_time.isna(), "model_create_time_ts"] = data_train.loc[~data_train.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.loc[~data_test.model_create_time.isna(), "model_create_time_ts"] = data_test.loc[~data_test.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

In [4]:
del data_train["create_time"]
del data_train["model_create_time"]
del data_test["create_time"]
del data_test["model_create_time"]

In [5]:
data_train.model_create_time_ts = data_train.model_create_time_ts.fillna(data_train.create_time_ts)
data_test.model_create_time_ts = data_test.model_create_time_ts.fillna(data_test.create_time_ts)

In [123]:
print("Рейтинг товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["rating_value"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["rating_value"].mean())

Рейтинг товаров, по которым позвонили:  1.5399876007439555 , по которым не звонили 1.4771690174232548


In [124]:
print("Длина описания товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["description_length"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["description_length"].mean())

Длина описания товаров, по которым позвонили:  572.92983638373 , по которым не звонили 573.338671921299


In [125]:
print("Представителей 0 класса: 67496, 1 класса: 37099")

Представителей 0 класса: 67496, 1 класса: 37099


In [15]:
A = data_train.groupby(['root_id'])['is_callcenter'].sum().sort_values()
B = data_train.groupby(['root_id'])['is_callcenter'].count().sort_values()
C = A.divide(B).sort_values()

order_id = list(data_train["order_id"].unique())
new_column = list()
for order in order_id:
    root = list(data_train[data_train["order_id"] == order]["root_id"].values)[0]
    new_column.append(C[root])
new_column_test = list()
for order in list(data_test["order_id"].unique()):
    root = list(data_test[data_test["order_id"] == order]["root_id"].values)[0]
    new_column_test.append(C[root])
data_train['root_mean_call'] = new_column
data_test['root_mean_call'] = new_column_test

In [27]:
new_column = list()
for order in range(data_train.shape[0]):
    hour = data_train["create_time_ts"][order].time().hour
    new_column.append(hour)
data_train['hour'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    hour = data_test["create_time_ts"][order].time().hour
    new_column.append(hour)
data_test['hour'] = new_column

In [54]:
data_train["utm_source"].fillna(0, inplace=True)
data_test["utm_source"].fillna(0, inplace=True)

In [66]:
data_train["rating_value"].fillna(0, inplace=True)
data_test["rating_value"].fillna(0, inplace=True)

In [75]:
data_train["rating_count"].fillna(0, inplace=True)
data_test["rating_count"].fillna(0, inplace=True)

In [149]:
new_column = list()
for order in range(data_train.shape[0]):
    day = data_train["create_time_ts"][order].date().weekday()
    new_column.append(day)
data_train['day_of_week'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    day = data_test["create_time_ts"][order].date().weekday()
    new_column.append(day)
data_test['day_of_week'] = new_column

In [164]:
data_train["life_time"] = (data_train.create_time_ts - data_train.model_create_time_ts).astype('timedelta64[D]')
data_test["life_time"] = (data_test.create_time_ts - data_test.model_create_time_ts).astype('timedelta64[D]')

In [203]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    max_time = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].max()
    new_column.append(max_time)
data_train['max_time_in_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    max_time = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].max()
    new_column.append(max_time)
data_test['max_time_in_page'] = new_column

In [205]:
data_train["max_time_in_page"].fillna(-1, inplace=True)
data_test["max_time_in_page"].fillna(-1, inplace=True)

In [206]:
features = ["pics_qty", 
            "goods_qty", 
            "description_length", 
            "is_moderated", 
            "model_id", 
            "parent_id", 
            "category_id", 
            "utm_medium", 
            "price", 
            "root_mean_call",
            "hour",
            "utm_source",
            "rating_value",
            "day_of_week",
            "life_time",
            "max_time_in_page"]

# ОБУЧЕНИЕ (TEST)

In [207]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import cv
from catboost import Pool

In [208]:
X_train_val, X_test, y_train_val, y_test = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.15, random_state=42, stratify=data_train["is_callcenter"])
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42, stratify = y_train_val)

In [209]:
model = CatBoostClassifier(eval_metric = 'AUC', class_weights = [0.35, 0.65], verbose = 100)

In [210]:
model.fit(X_train, y_train,eval_set=(X_val, y_val), use_best_model=True)

Learning rate set to 0.092227
0:	test: 0.6902268	best: 0.6902268 (0)	total: 16.5ms	remaining: 16.4s
100:	test: 0.7233423	best: 0.7233423 (100)	total: 1.05s	remaining: 9.38s
200:	test: 0.7248600	best: 0.7248974 (193)	total: 2.02s	remaining: 8.04s
300:	test: 0.7255366	best: 0.7257080 (274)	total: 3.03s	remaining: 7.04s
400:	test: 0.7246319	best: 0.7257080 (274)	total: 4.03s	remaining: 6.01s
500:	test: 0.7241534	best: 0.7257080 (274)	total: 4.97s	remaining: 4.95s
600:	test: 0.7238006	best: 0.7257080 (274)	total: 6s	remaining: 3.98s
700:	test: 0.7229794	best: 0.7257080 (274)	total: 6.97s	remaining: 2.97s
800:	test: 0.7229060	best: 0.7257080 (274)	total: 7.94s	remaining: 1.97s
900:	test: 0.7223848	best: 0.7257080 (274)	total: 8.96s	remaining: 985ms
999:	test: 0.7219336	best: 0.7257080 (274)	total: 9.93s	remaining: 0us

bestTest = 0.7257079603
bestIteration = 274

Shrink model to first 275 iterations.


In [211]:
predictions = model.predict(X_test[features])

In [212]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predictions)

0.6611111776646367

# ОБУЧЕНИЕ (Kaggle)

In [213]:
X_train, X_val, y_train, y_val = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.2, random_state=42, stratify=data_train["is_callcenter"])

In [214]:
model = CatBoostClassifier(eval_metric = 'AUC', class_weights = [0.35, 0.65], verbose = 100)
model.fit(X_train, y_train,eval_set=(X_val, y_val), use_best_model=True)

Learning rate set to 0.094578
0:	test: 0.6956632	best: 0.6956632 (0)	total: 18.2ms	remaining: 18.2s
100:	test: 0.7241020	best: 0.7241360 (96)	total: 1.28s	remaining: 11.4s
200:	test: 0.7260347	best: 0.7261765 (197)	total: 2.5s	remaining: 9.93s
300:	test: 0.7266476	best: 0.7266705 (299)	total: 3.8s	remaining: 8.82s
400:	test: 0.7269057	best: 0.7270301 (340)	total: 4.87s	remaining: 7.27s
500:	test: 0.7268669	best: 0.7271251 (481)	total: 5.91s	remaining: 5.88s
600:	test: 0.7260575	best: 0.7271251 (481)	total: 6.95s	remaining: 4.61s
700:	test: 0.7258571	best: 0.7271251 (481)	total: 7.98s	remaining: 3.4s
800:	test: 0.7258402	best: 0.7271251 (481)	total: 9.05s	remaining: 2.25s
900:	test: 0.7255495	best: 0.7271251 (481)	total: 10.1s	remaining: 1.11s
999:	test: 0.7249904	best: 0.7271251 (481)	total: 11.1s	remaining: 0us

bestTest = 0.72712513
bestIteration = 481

Shrink model to first 482 iterations.


In [215]:
predictions = model.predict(data_test[features])

In [216]:
id_column = data_test["order_id"]
result_column = predictions
dataset = pd.DataFrame({'order_id': id_column, 'is_callcenter': list(result_column)}, columns=['order_id', 'is_callcenter'])
dataset.to_csv("sample_submission.csv", index=False)